# ML Modeling with Iris Dataset and Sklearn

### Goals

* Traing ML Model
* Create Python Wrapper with predict function
* Build docker image (s2i) with Wrapper and model


In [1]:
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets
import numpy as np
import joblib
from joblib import dump, load
import seldon_core

In [40]:
sklearn.__version__

'0.24.2'

In [45]:
joblib.__version__

'1.0.1'

In [47]:
seldon_core.__version__

'1.10.0'

In [4]:
iris = datasets.load_iris()

In [7]:
iris['data'].shape

(150, 4)

In [8]:
iris['target'].shape

(150,)

In [13]:
np.unique(iris['target'])

array([0, 1, 2])

In [32]:
X = iris['data']
y = iris['target']

In [34]:
X[0]

array([5.1, 3.5, 1.4, 0.2])

In [35]:
clf = RandomForestClassifier(random_state=42)

In [36]:
clf.fit(X, y)

RandomForestClassifier(random_state=42)

In [38]:
clf.score(X, y)

1.0

In [39]:
dump(clf, "model.joblib")

['model.joblib']

## Testing Deployment Python Wrapper

In [2]:
cat Model.py

# predict function modified from https://docs.seldon.io/projects/seldon-core/en/latest/workflow/github-readme.html

from joblib import dump, load
import logging

logging.basicConfig(format='%(asctime)s (%(levelname)s) %(message)s',
                    level=logging.DEBUG,
                    datefmt='%d.%m.%Y %H:%M:%S')

class Model:
    def __init__(self):
        self._model = load("model.joblib")

    def predict(self, X, features_names=None):

        logging.info(f'received X {X}')

        output = self._model.predict(X)

        logging.info(f"model output {output}")

        return output

    def health_status(self):
        # check if model is working
        response = self.model.predict([1, 2], ["f1", "f2"])

        # simple model validation
        assert len(response) == 2, "health check returning bad predictions" 

        return response

    def tags(self,X):

        # adding fixed tag to returned metadata
        return {"system":"dev"}

In [3]:
from Model import Model

In [4]:
Model().predict([[1,2,3,4]])

01.09.2021 11:01:08 (INFO) received X [[1, 2, 3, 4]]
01.09.2021 11:01:08 (INFO) model output [2]


array([2])

### Building Image

In [1]:
! make build

## Run docker locally and Test API access

In [2]:
! docker run --rm -d -p 9000:9000/tcp sklearn_iris2:0.1

2581bdee76b792a333e5b261ed2fcedc867c9acd1cb4ad17212dfc837d1c456a


Tag is returned in meta field:

In [5]:
! cat api_access_curl.sh;

# port 9000 exposed through the sklearn container
curl -s -X POST http://localhost:9000/api/v1.0/predictions \
    -H 'Content-Type: application/json' \
    -d '{ "data": { "ndarray": [[1,2,3,4]] } }'

In [9]:
!./api_access_curl.sh

{"data":{"names":["proba"],"ndarray":[[0.39731466202150834]]},"meta":{"requestPath":{"classifier":"seldonio/mock_classifier:1.6.0-dev"}}}


### Checking Health STatus

In [10]:
! curl localhost:9000/health/ping

404 page not found


In [11]:
! curl localhost:9000/health/status

404 page not found


## Push to Registry

Push to the registry of your choice (content omitted)

## K8s Access

(portforwarding)

In [38]:
! cat api_access_curl.sh; ./api_access_curl.sh

# port 9000 exposed through the sklearn container
curl -s -X POST http://localhost:9000/api/v1.0/predictions \
    -H 'Content-Type: application/json' \
    -d '{ "data": { "ndarray": [[1,2,3,4]] } }'

# port 8000 exposed through seldon-container-engine container
# Swagger API doc at http://localhost:8000/api/v1.0/doc/

{"data":{"names":[],"ndarray":[2]},"meta":{"requestPath":{"classifier-custom":"chritter/sklearn_iris:0.4"}}}
